# Customize gate calibrations

The pulse implementation of a gate is called "gate calibration". Gaining access to gate calibrations empowers you to experiment with quantum gate designs and execute quantum programs with customized, improved gate calibrations. This notebook shows you how to access the native gate calibrations provided by quantum hardware providers, and shows you how to define custom gate calibrations for your quantum programs.

We start with basic imports, and initialize the Rigetti Aspen-M-3 device which is the targeted device for this example notebook.

In [1]:
import numpy as np

import braket.experimental.autoqasm as aq
from braket.experimental.autoqasm import pulse
from braket.experimental.autoqasm.instructions import rx, rz, measure

from braket.aws import AwsDevice
from braket.devices import Devices
from braket.circuits import Gate, QubitSet, FreeParameter
from braket.pulse import DragGaussianWaveform

device = AwsDevice(Devices.Rigetti.AspenM3)

## Gate calibrations from hardware providers

To customize a gate calibration, you often need to be intimately familiar with how quantum gates are implemented with pulses. It is often a good place to start by viewing the native gate calibrations supplied by hardware providers. To do so, we first retrieve the information from the `gate_calibrations` attribute of a device. 

In [2]:
provider_calibrations = device.gate_calibrations

Even for the same gate, different physical qubits and angles usually have a different pulse implementation. Because of this, a gate calibration may be associated with specific physical qubits and angles. In the code snippet below, we retrieve the gate calibration of the Rx gate for physical qubit `$0` and `angle=pi/2`.

In [3]:
pulse_sequence_rx_pi_2_q0 = provider_calibrations.pulse_sequences[(Gate.Rx(np.pi / 2), QubitSet(0))]

print(pulse_sequence_rx_pi_2_q0.to_ir())

OPENQASM 3.0;
cal {
    waveform wf_drag_gaussian_1 = drag_gaussian(24.0ns, 2.547965400864ns, 2.3154335273287345e-10, 0.294418024251374, false);
    barrier $0;
    shift_frequency(q0_rf_frame, -321047.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321047.14178613486);
    barrier $0;
}


Some gate calibrations are defined not with fixed values of angles, but rather with variables. In this case, these gate calibrations can be retrieved with angles of `FreeParameter` type. For example, the code snippet below retrieves the gate calibration of the Rz gate on the qubit `$1` with a variable angle `theta`.

In [4]:
theta = FreeParameter("theta")
pulse_sequence_rz_theta_q0 = provider_calibrations.pulse_sequences[(Gate.Rz(theta), QubitSet(1))]

print(pulse_sequence_rz_theta_q0.to_ir())

OPENQASM 3.0;
cal {
    barrier $1;
    shift_phase(q1_rf_frame, -1.0*theta);
    shift_phase(q0_q1_xy_frame, 0.5*theta);
    shift_phase(q1_q2_xy_frame, 0.5*theta);
    shift_phase(q1_q16_xy_frame, 0.5*theta);
    barrier $1;
}


## Compose custom gate calibrations with AutoQASM
In this section, we show you how to customize a gate calibration and use it in a quantum program. To create a custom gate calibration, you need to 
1. Find the definition of the gate in AutoQASM.
2. Decide the qubits and angles you want to define the calibration.
3. Compose a pulse program that defines the calibration.

As an example, let's define the gate calibration of `rx` on qubit `$1` and at angle `pi/2`. We first inspect the definition of the `rx` gate. You can find the definition of gate instructions in the [gate module](https://github.com/amazon-braket/amazon-braket-sdk-python/blob/feature/autoqasm/src/braket/experimental/autoqasm/instructions/gates.py) of AutoQASM. You can also use the Python built-in `help` function to retrieve the definition. A gate calibration for the `rx` gate must fully specify the inputs of the gate, `target` and `angle`.

In [5]:
help(rx)

Help on function rx in module braket.experimental.autoqasm.instructions.gates:

rx(target: Union[int, oqpy.classical_types._ClassicalVar, oqpy.base.OQPyExpression, str, oqpy.quantum_types.Qubit], angle: float) -> None
    X-axis rotation gate.
    
    Args:
        target (QubitIdentifierType): Target qubit.
        angle (float): Rotation angle in radians.



To specify fixed values for the parameters `target` and `angle`, we set the values through keyword arguments of the decorator. The pulse implementation for the gate calibration is in the body of the `my_rx_cal` function decorated by `@aq.gate_calibration`. In the example in the next cell, we want to experiment with offsetting the frequency of the pulse by 100 Hz away from the hardware provider's implementation. The gate calibration `my_rx_cal` recreates the hardware provider's implementation but with an offset in the frequency.

In [6]:
wf = DragGaussianWaveform(
    24e-9, 2.547965400864e-9, 2.370235498840002e-10, 0.293350447987059, False, "wf_drag_gaussian_1"
)
q0_rf_frame = device.frames['q0_rf_frame']

@aq.gate_calibration(implements=rx, target="$0", angle=np.pi/2)
def my_rx_cal():
    pulse.barrier("$0")
    pulse.shift_frequency(q0_rf_frame, -321047.14178613486 - 100)
    pulse.play(q0_rf_frame, wf)
    pulse.shift_frequency(q0_rf_frame, 321047.14178613486 + 100)
    pulse.barrier("$0")

Next, we will demonstrate how to attach `my_rx_cal` to the main quantum program `main_program`. The program is defined in the code block below.

In [7]:
@aq.main
def my_program():
    rx("$0", np.pi/2)
    rz("$1", 0.123)
    measure("$0")

main_program = my_program()
print(main_program.to_ir())

OPENQASM 3.0;
rx(1.5707963267948966) $0;
rz(0.123) $1;
bit __bit_0__;
__bit_0__ = measure $0;


To attach gate calibrations to the program, call the `with_calibrations` method on `main_program` to create a new program with `my_rx_cal` attached, leaving the original intact. This allows you to experiment with different gate calibrations on the same program without the need to redefine the main program every time. 

In the printed OpenQASM script of the program, the gate definition for the `rx` gate becomes a `defcal` block.

In [8]:
custom_program = main_program.with_calibrations(my_rx_cal)
print(custom_program.to_ir())

OPENQASM 3.0;
cal {
    waveform wf_drag_gaussian_1 = drag_gaussian(24.0ns, 2.547965400864ns, 2.370235498840002e-10, 0.293350447987059, false);
}
defcal rx(1.5707963267948966) $0 {
    barrier $0;
    shift_frequency(q0_rf_frame, -321147.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321147.14178613486);
    barrier $0;
}
rx(1.5707963267948966) $0;
rz(0.123) $1;
bit __bit_0__;
__bit_0__ = measure $0;


You can also define a gate calibration with variable parameters. Variable parameters are used in the body of the decorated function. The variable parameters must be arguments of the decorated Python function, instead of being arguments to the decorator `@aq.gate_calibration`. The variable parameters must have a type hint of either `aq.Qubit` for qubits or `float` for angles. Let's define another gate calibration with a variable parameter. This time, we define a calibration for the `rz` gate on qubit `$1` and a variable angle `angle`.

In [9]:
q1_rf_frame = device.frames['q1_rf_frame']

@aq.gate_calibration(implements=rz, target="$1")
def my_rz_cal(angle: float):
    pulse.barrier("$1")
    pulse.shift_frequency(q1_rf_frame, -1.0*angle)
    pulse.barrier("$1")

We then attach this gate calibration, `my_rz_cal`, to the main program we composed previously, together with the other gate calibration, `my_rx_cal`.

In [10]:
custom_program = main_program.with_calibrations([my_rx_cal, my_rz_cal])
print(custom_program.to_ir())

OPENQASM 3.0;
cal {
    waveform wf_drag_gaussian_1 = drag_gaussian(24.0ns, 2.547965400864ns, 2.370235498840002e-10, 0.293350447987059, false);
}
defcal rx(1.5707963267948966) $0 {
    barrier $0;
    shift_frequency(q0_rf_frame, -321147.14178613486);
    play(q0_rf_frame, wf_drag_gaussian_1);
    shift_frequency(q0_rf_frame, 321147.14178613486);
    barrier $0;
}
defcal rz(angle[32] angle) $1 {
    barrier $1;
    shift_frequency(q1_rf_frame, -1.0 * angle);
    barrier $1;
}
rx(1.5707963267948966) $0;
rz(0.123) $1;
bit __bit_0__;
__bit_0__ = measure $0;


In the printed OpenQASM script of the program, the gate definitions for the `rx` gate and the `rz` become `defcal` blocks of the program. The variable parameter `angle` for the `rz` gate is captured.

## Summary
This example notebook shows you how to retrieve and view the gate calibrations supplied by hardware providers. With AutoQASM, you can customize gate calibrations to explore different pulse implementations of gates. You can define gate calibrations against fixed values of gate parameters, as well as using parametric definitions for variable parameters. Multiple gate calibrations can be attached to a program to create a new program. This makes it easy to experiment with different gate calibrations on a single program.